In [1]:
first = True
get_base_data = False
from google.colab import drive
drive.mount('/content/drive')
drivef = "/content/drive/MyDrive/"
if first:
  !pip install visdom
  !pip install open3d
  !pip install ninja
  if True:
    !mkdir /content/repo_folder
    !rm -rf /content/MSN-Point-Cloud-Completion
    %cd /content/repo_folder
    !git clone https://github.com/iwillnotkillyou/MSN-Point-Cloud-Completion.git
    %cd ./MSN-Point-Cloud-Completion
    !git checkout Pavel
  !git clone https://github.com/alexandrosstergiou/SoftPool.git

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=1448350efd792a0bb4e14974c0d3f5d440063b1f313c1f8ff838dbc9e8a041c2
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully u

In [2]:
from pathlib import Path
import os
import torch
if False:
  olf = "SoftPool/pytorch/CUDA"
  newf = "softpool_cuda"
  tempn = "tempnamed"
  os.rename(newf, tempn)
  !cp -r $olf ./
  os.rename("CUDA", newf)
  n = "soft_pool_cuda_module.py"
  copiedfn = str(Path(tempn) / n)
  destfn = str(Path(newf) / n)
  !cp $copiedfn $destfn
  !rm -r "CUDA"
  !rm -r $tempn
  from softpool_cuda.soft_pool_cuda_module import soft_pool3d, SoftPool3d
  r = SoftPool3d()(torch.ones(100,3,100,100,100))
  print(r.shape)

In [3]:
import os
dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
if first:
  if get_base_data:
    !unzip -o /content/drive/MyDrive/val.zip -d $dataf
    !unzip -o /content/drive/MyDrive/complete.zip -d $dataf
    listp = f'{dataf}/train.list'
    valp = f'{dataf}/val.list'
    listop = f'{drivef}/train.list'
    valop = f'{drivef}/val.list'
    !cp $listop $listp
    !cp $valop $valp
  %cd /content/repo_folder/MSN-Point-Cloud-Completion
  !mkdir "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model"
  !cp "/content/drive/MyDrive/network.pth" "/content/repo_folder/MSN-Point-Cloud-Completion/trained_model/network.pth"

/content/repo_folder/MSN-Point-Cloud-Completion


In [4]:
if False:
  dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  print('Downloading ...')
  !wget http://kaldir.vc.in.tum.de/cdiller/ShapeNetPointClouds.zip -P $dataf
  print('Extracting ...')
  fp = dataf + '/ShapeNetPointClouds.zip'
  !unzip -o -q $fp -d $dataf
  !rm $fp
  print('Done.')

In [5]:
if False:
  dataf = "/content/repo_folder/MSN-Point-Cloud-Completion/data"
  fp = dataf + '/dataset.zip'
  !wget https://cloud.tsinghua.edu.cn/f/06a3c383dc474179b97d/?dl=1 -O $fp
  !unzip -o -q $fp -d $dataf

In [6]:
if False:
  !python val.py

In [7]:
%cd /content/repo_folder/MSN-Point-Cloud-Completion
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import sys
import expansion_penalty.expansion_penalty_module as expansion
import MDS.MDS_module as MDS_module
from model import *

class LocallyConnected1d():
    def __init__(self, in_channels,out_channels,output_size, bias = False, kernel_size = 1, stride = 1):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.weight = nn.Parameter(
            torch.randn(output_size, out_channels, in_channels * kernel_size)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(out_channels, output_size)
            )
        else:
            self.register_parameter('bias', None)
    def forward(self,x):
        x = x.unfold(1, self.kernel_size, self.stride)
        out = torch.matmul(x.unsqueeze(2), self.weight).squeeze()
        if self.bias is not None:
            out += self.bias
        return out

class SimpleLocallyConnected1d():
    def __init__(self, in_channels,out_channels,output_size, bias = False):
        super().__init__()
        self.weight = nn.Parameter(
            torch.randn(output_size, out_channels, in_channels)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(output_size, out_channels)
            )
        else:
            self.register_parameter('bias', None)
    def forward(self,x):
        out = torch.matmul(x.unsqueeze(2),self.weight).squeeze()
        if self.bias is not None:
            out += self.bias
        return out

class ResSizes(nn.Module):
    def __init__(self, sizesup, sizesdown):
        super().__init__()


        lsup = []
        for i in range(len(sizesup)-1):
            lsup.append(BatchNormConv1D(sizesup[i],sizesup[i+1]))
        lsdown = []
        for i in range(len(sizesdown)-1):
            lsdown.append(BatchNormConv1D(sizesdown[i] + (0 if i != 0 else sizesup[0]),sizesdown[i+1]))
        self.sizesdown = sizesdown
        self.sizesup = sizesup
        self.convsup = torch.nn.Sequential(*lsup)
        self.convsdown = torch.nn.Sequential(*lsdown)
        self.conv1 = BatchNormConv1D(4, sizesup[0])
        self.convlast = torch.nn.Conv1d(sizesdown[-1], 3, 1)
        self.bnlast = torch.nn.BatchNorm1d(3)
        self.th = nn.Tanh()

    def forward(self, x):
        batchsize = x.size()[0]
        npoints = x.size()[2]
        x = self.conv1(x)
        pointfeat = x
        x = self.convsup(x)
        s = x.size()[1]
        softmaxweights = F.softmax(x,2)
        x = (softmaxweights*x).sum(2)
        x = x.view(batchsize, -1, 1)
        x = x.broadcast_to(x.shape[0], x.shape[1], npoints)
        x = torch.cat([x, pointfeat], 1)
        x = self.convsdown(x)
        x = self.th(self.bnlast(self.convlast(x)))
        return x

class BatchNormLocalConv1D(nn.Module):
    def __init__(self, in_channels,out_channels,output_size,kernel_size = 1, stride = 1):
        super().__init__()
        self.conv = LocallyConnected1d(in_channels,out_channels,output_size,False,kernel_size,stride)
        self.bn = torch.nn.BatchNorm1d(out_channels)

    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

class BatchNormConv1D(nn.Module):
    def __init__(self, in_channels,out_channels,kernel_size = 1, stride = 1):
        super(BatchNormConv1D, self).__init__()
        self.conv = nn.Conv1d(in_channels,out_channels,kernel_size,stride)
        self.bn = torch.nn.BatchNorm1d(out_channels)

    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

class BatchNormConv1DTransformer(nn.Module):
    def __init__(self, in_channels,out_channels,kernel_size = 1, stride = 1):
        super(BatchNormConv1DTransformer, self).__init__()
        self.conv = nn.Conv1d(in_channels,out_channels,kernel_size,stride)
        self.bn = torch.nn.BatchNorm1d(out_channels)

    def forward(self, _, __, x):
        return F.relu(self.bn(self.conv(x)))

def make(sizes,f,additional_args = None):
    return [f(*([sizes[i],sizes[i+1]]+ [] if additional_args is None or additional_args[i] is None else additional_args[i])) for i in range(len(sizes)-1)]

class GlobalTransform(nn.Module):
    def __init__(self, bottleneck_size, sizes, latents):
        super().__init__()
        self.bottleneck_size = bottleneck_size
        self.latents = latents
        sizes = (3,) + tuple(sizes) + (self.latents*self.latents,)
        self.convs = nn.Sequential(*make(sizes, lambda x,y : nn.Sequential(BatchNormConv1D(x,y))))
        self.register_buffer('identity', torch.diag(torch.ones(self.latents)))


    def forward(self, partial, x):
        bs = x.shape[0]
        x = x.view(bs,-1,self.latents).contiguous()
        outs = []
        transform_pre = self.convs(partial)
        softmaxweights = F.softmax(transform_pre,2)
        transform = (softmaxweights*transform_pre).sum(2).view(-1, self.latents, self.latents)
        identity = torch.broadcast_to(self.identity.unsqueeze(0),(bs, self.identity.shape[0], self.identity.shape[1]))
        transform = transform + identity
        for i in range(x.shape[1]):
          outs.append(torch.matmul(transform, x[:,i,:].unsqueeze(2)).squeeze())
        return torch.cat(outs,1)

class GlobalTransformIndentity(nn.Module):
    def __init__(self):
        super().__init__()


    def forward(self, partial, x):
        return x

def f(partial,x,expansion,num_points,n_primitives,residual,decoder):
  outs = []
  for i in range(0, n_primitives):
    rand_grid = Variable(torch.cuda.FloatTensor(x.size(0), 2, num_points // n_primitives))
    rand_grid.data.uniform_(0, 1)
    y = x.unsqueeze(2).expand(x.size(0), x.size(1), rand_grid.size(2)).contiguous()
    y = torch.cat((rand_grid, y), 1).contiguous()
    outs.append(decoder[i](y))


  outs = torch.cat(outs, 2)
  out1 = outs.transpose(1, 2).contiguous()
  dist, _, mean_mst_dis = expansion(out1, num_points // n_primitives, 1.5)
  loss_mst = torch.mean(dist)

  id0 = torch.zeros(outs.shape[0], 1, outs.shape[2], device=torch.device('cuda')).contiguous()
  outs = torch.cat((outs, id0), 1)
  id1 = torch.ones(partial.shape[0], 1, partial.shape[2], device=torch.device('cuda')).contiguous()
  partial = torch.cat((partial, id1), 1)
  xx = torch.cat((outs, partial), 2)

  resampled_idx = MDS_module.minimum_density_sample(xx[:, 0:3, :].transpose(1, 2).contiguous(), out1.shape[1],mean_mst_dis)
  xx = MDS_module.gather_operation(xx, resampled_idx)
  delta = residual(xx)
  xx = xx[:, 0:3, :]
  out2 = (xx + delta).transpose(2, 1).contiguous()
  return out1, out2, loss_mst



class TransformMSN(nn.Module):
    def __init__(self, residual, additional_encoder,
                 num_points=8192, n_primitives=16):
        super().__init__()
        self.num_points = num_points
        self.bottleneck_size = 1024
        self.n_primitives = n_primitives
        self.encoder = nn.Sequential(
            PointNetfeat(num_points),
            nn.Linear(1024, self.bottleneck_size),
            nn.BatchNorm1d(self.bottleneck_size),
            nn.ReLU()
        )
        self.decoder = nn.ModuleList(
            [PointGenCon(bottleneck_size=2 + self.bottleneck_size) for i in range(0, self.n_primitives)])
        self.additional_encoder = additional_encoder(self.bottleneck_size)
        self.res = residual()
        self.expansion = expansion.expansionPenaltyModule()

    def freeze(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        for param in self.decoder.parameters():
            param.requires_grad = False
        self.res.freeze()

    def forward(self, x):
        partial = x
        x = self.encoder(x)
        x = self.additional_encoder(partial,x)
        return f(partial,x,self.expansion,self.num_points,self.n_primitives,self.res,self.decoder)

class TransformMSN2(nn.Module):
    def __init__(self, residual, additional_encoder,
                 num_points=8192, n_primitives=16):
        super().__init__()
        self.num_points = num_points
        self.bottleneck_size = 1024
        self.n_primitives = n_primitives
        self.encoder = nn.Sequential(
            PointNetfeat(num_points),
            nn.Linear(1024, self.bottleneck_size),
            nn.BatchNorm1d(self.bottleneck_size),
            nn.ReLU()
        )
        self.decoder = nn.ModuleList(
            [PointGenCon(bottleneck_size=2 + self.bottleneck_size) for i in range(0, self.n_primitives)])
        print(self.decoder[0].conv3.out_channels)
        self.additional_encoder = additional_encoder(self.bottleneck_size)
        self.residual = residual()
        self.expansion = expansion.expansionPenaltyModule()

    def freeze(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        for param in self.decoder.parameters():
            param.requires_grad = False

    def forward(self, x):
        partial = x
        x = self.encoder(x)
        x = x + self.additional_encoder(partial)
        return f(partial,x,self.expansion,self.num_points,self.n_primitives,self.residual,self.decoder)

/content/repo_folder/MSN-Point-Cloud-Completion


In [8]:
import torch
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import os
import random
#from utils import *

def resample_pcd(pcd, n):
    """Drop or duplicate points so that pcd has exactly n points"""
    idx = np.random.permutation(pcd.shape[0])
    if idx.shape[0] < n:
        idx = np.concatenate([idx, np.random.randint(pcd.shape[0], size = n - pcd.shape[0])])
    return pcd[idx[:n]]

class ShapeNet(data.Dataset):
    def __init__(self, train = True, npoints = 8192):
        if train:
            self.list_path = './data/train.list'
        else:
            self.list_path = './data/val.list'
        self.npoints = npoints
        self.train = train

        with open(os.path.join(self.list_path)) as file:
            self.model_list = [line.strip().replace('/', '_') for line in file]
        random.shuffle(self.model_list)
        self.len = len(self.model_list * 50)

    def __getitem__(self, index):
        model_id = self.model_list[index // 50]
        scan_id = index % 50
        def read_pcd(filename):
            pcd = o3d.io.read_point_cloud(filename)
            return torch.from_numpy(np.array(pcd.points)).float()
        if self.train:
            partial = read_pcd(os.path.join("./data/train/", model_id + '_%d_denoised.pcd' % scan_id))
        else:
            partial = read_pcd(os.path.join("./data/val/", model_id + '_%d_denoised.pcd' % scan_id))
        complete = read_pcd(os.path.join("./data/complete/", '%s.pcd' % model_id))
        return model_id, resample_pcd(partial, 5000), resample_pcd(complete, self.npoints)

    def __len__(self):
        return self.len

class EmbeddingsDataset(torch.utils.data.Dataset):
    def __init__(self, folder, embedder_batch_size, transform=None):
        self.folder = folder
        self.files = os.listdir(folder)
        self.transform = transform
        self.embedder_batch_size = embedder_batch_size

    def __len__(self):
        return len(self.files*self.embedder_batch_size)

    def __getitem__(self, idx):
        id, x, target = torch.load(f"{self.folder}/{self.files[idx // self.embedder_batch_size]}")
        i = idx % self.embedder_batch_size
        try:
          r = id[i], (x[0][i], x[1][i]), target[i]
        except:
          print(x[0].shape, x[1].shape)
        return r

In [9]:
class FullModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.EMD = emd.emdModule()

    def forward(self, inputs, gt, eps, iters):
        output1, output2, expansion_penalty = self.model(inputs)
        gt = gt[:, :, :3]
        idx = np.random.randint(gt.shape[1], self.model.num_points)
        gt = gt[:, idx, :]
        print(gt.shape)
        print(output2.shape)
        dist, _ = self.EMD(output1, gt, eps, iters)
        emd1 = torch.sqrt(dist).mean(1)

        dist, _ = self.EMD(output2, gt, eps, iters)
        emd2 = torch.sqrt(dist).mean(1)

        return output1, output2, emd1, emd2, expansion_penalty

class PointNetResSoftMax(nn.Module):
    def __init__(self, sizes):
        super().__init__()
        sizes = (1088,) + tuple(sizes)
        self.conv1 = torch.nn.Conv1d(4, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(1024)
        self.convs = nn.Sequential(*make(sizes, lambda x,y : BatchNormConv1D(x,y)))
        self.lastconv = torch.nn.Conv1d(sizes[-1], 3, 1)
        self.th = nn.Tanh()

    def freeze(self):
        for param in self.parameters():
          param.requires_grad = False
        for param in self.convs.parameters():
          param.requires_grad = True
        for param in self.lastconv.parameters():
          param.requires_grad = True

    def forward(self, x):
        batchsize = x.size()[0]
        npoints = x.size()[2]
        x = F.relu(self.bn1(self.conv1(x)))
        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        softmaxweights = F.softmax(x,2)
        x = (softmaxweights*x).sum(2)
        x = x.view(-1, 1024)
        x = x.view(-1, 1024, 1).repeat(1, 1, npoints)
        x = torch.cat([x, pointfeat], 1)
        x = self.convs(x)
        x = self.th(self.lastconv(x))
        return x

def make_fuller_model(name, args, architect_args):
  def gtFunc(x):
    return GlobalTransform(x,architect_args.gt_layer_sizes,architect_args.latents) if architect_args.use_gt else GlobalTransformIndentity()
  def residualFunc():
    return PointNetResSoftMax(architect_args.res_layer_sizes )
  model = TransformMSN(residual = residualFunc, additional_encoder = gtFunc)
  network = FullModel(model)
  #network = torch.nn.DataParallel(network)
  #model.apply(weights_init) #initialization of the weight

  if name != '':
      previous_state_dict = torch.load(name)
      network.load_state_dict(previous_state_dict,True,True)
      print("Previous weight loaded ")
  elif args.base_model != '':
      previous_state_dict = torch.load(args.base_model)
      model.load_state_dict(previous_state_dict,False,True)
      print("Previous weight loaded ")
  model.freeze()
  return network
! mkdir /content/saved
! wget -nc -O /content/saved/network.pth https://www.dropbox.com/scl/fi/qx16y8hkfaq0b7nh1k4b2/network.pth?rlkey=w6yjdwy3zpbf25h2qtak6kkq5&dl=0

--2024-01-31 14:07:15--  https://www.dropbox.com/scl/fi/qx16y8hkfaq0b7nh1k4b2/network.pth?rlkey=w6yjdwy3zpbf25h2qtak6kkq5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7358ae52eb413bab999059e959.dl.dropboxusercontent.com/cd/0/inline/CMZ4VriKAAwecl7GjQd8KamIrdyCq5lWl-yEbcxvI_RdZ2pmSa_2Z0KSEUchrO0dBSDGr18Q5wmmwTc8NEW0zhdpFMhjv-fC7VJTQVhiK7QKYHugrAYG5Vl59gVwUUa6kZVaI_5K8eWHgjYCFx5iRx0j/file# [following]
--2024-01-31 14:07:16--  https://uc7358ae52eb413bab999059e959.dl.dropboxusercontent.com/cd/0/inline/CMZ4VriKAAwecl7GjQd8KamIrdyCq5lWl-yEbcxvI_RdZ2pmSa_2Z0KSEUchrO0dBSDGr18Q5wmmwTc8NEW0zhdpFMhjv-fC7VJTQVhiK7QKYHugrAYG5Vl59gVwUUa6kZVaI_5K8eWHgjYCFx5iRx0j/file
Resolving uc7358ae52eb413bab999059e959.dl.dropboxusercontent.com (uc7358ae52eb413bab999059e959.dl.dropboxusercontent.com)... 162.125.81.15, 262

In [10]:
%cd /content/repo_folder/MSN-Point-Cloud-Completion
#! git fetch origin
#! git reset --hard origin/master
%load_ext autoreload
%autoreload 2
import open3d as o3d
import argparse
import random
import numpy as np
import torch
import torch.optim as optim
import sys
from model import *
from utils import *
import os
import json
import time, datetime
from time import time
from torch.utils.cpp_extension import load
import emd.emd_module as emd
from tqdm import tqdm

torch.cuda.empty_cache()
import gc
gc.collect()

def trainFull(network, dir_name, val_only, args):
  lrate = 0.001 #learning rate
  optimizer = optim.Adam(model.parameters(), lr = lrate)
  train_curve = []
  val_curve = []
  labels_generated_points = (torch.Tensor(range(1, (args.n_primitives+1)*(args.num_points//args.n_primitives)+1))
  .view(args.num_points//args.n_primitives,(args.n_primitives+1)).transpose(0,1))
  labels_generated_points = (labels_generated_points)%(args.n_primitives+1)
  labels_generated_points = labels_generated_points.contiguous().view(-1)
  print("Random Seed: ", args.manualSeed)
  random.seed(args.manualSeed)
  torch.manual_seed(args.manualSeed)
  best_val_loss = 10
  network.to("cuda")
  x = ShapeNet(train=False, npoints=args.num_points)
  perc_train = (1 - args.perc_val_data)*args.perc_data
  perc_val = args.perc_val_data*args.perc_data
  dataset, dataset_val, _ = torch.utils.data.random_split(x,[perc_train,perc_val,1-(perc_train+perc_val)])
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batchSize,
                                            shuffle=True, num_workers = args.workers,drop_last=True)
  dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=args.batchSize,
                                            shuffle=False, num_workers = args.workers, drop_last=True)

  len_dataset = len(dataset)
  len_val_dataset = len(dataset_val)
  print("Train Set Size: ", len_dataset)
  try:
    for epoch in range(args.nepoch):
        train_loss = np.zeros(len(dataloader))
        val_loss = np.zeros(len(dataloader_val))
        #TRAIN MODE
        model.train()

        # learning rate schedule
        if epoch==20:
            optimizer = optim.Adam(model.parameters(), lr = lrate/10.0)
        if epoch==40:
            optimizer = optim.Adam(model.parameters(), lr = lrate/100.0)

        if not val_only:
          for i, data in enumerate(dataloader, 0):
              if args.epoch_iter_limit is not None and i > args.epoch_iter_limit:
                break
              optimizer.zero_grad()
              id, input, gt = data
              input = input.float().cuda().transpose(1,2)
              gt = gt.float().cuda()

              output1, output2, emd1, emd2, expansion_penalty  = network(input, gt.contiguous(), 0.005, 10)
              emd1m = emd1.mean()
              emd1mi = emd1m.item()
              emd2m = emd2.mean()
              emd2mi = emd2m.item()
              exppm = expansion_penalty.mean()
              exppmi = exppm.item()
              loss_net = emd1m + emd2m+ exppm * 0.1
              train_loss[i] = emd2mi
              loss_net.backward()
              optimizer.step()

              print(args.env + ' train [%d: %d/%d]  emd1: %f emd2: %f expansion_penalty: %f'
                    %(epoch, i, len_dataset/args.batchSize, emd1mi, emd2mi,
                      exppmi))
              if (i*args.batchSize) % 100 == 0:
                if not os.path.exists(dir_name):
                  os.makedirs(dir_name)
                torch.save(model.state_dict(), '%s/network.pth' % (dir_name))
              del(input)
          train_curve.append(np.mean(train_loss))

        # VALIDATION
        if epoch % 2 == 0:
            model.eval()
            with torch.no_grad():
                for i, data in enumerate(dataloader_val):
                    if args.epoch_iter_limit_val is not None and i > args.epoch_iter_limit_val:
                      break
                    id,input, gt = data
                    input = input.float().cuda().transpose(1,2)
                    gt = gt.float().cuda()
                    output1, output2, emd1, emd2, expansion_penalty  = network(input, gt.contiguous(), 0.004, 100 if val_only else 20)
                    emd1m = emd1.mean()
                    emd1mi = emd1m.item()
                    emd2m = emd2.mean()
                    emd2mi = emd2m.item()
                    val_loss[i] = emd2mi
                    exppm = expansion_penalty.mean()
                    exppmi = exppm.item()
                    idx = random.randint(0, input.size()[0] - 1)
                    print(args.env + ' val [%d: %d/%d]  emd1: %f emd2: %f expansion_penalty: %f'
                          %(epoch, i, len_val_dataset/args.batchSize, emd1mi,
                            emd2mi, exppmi))
                    del(input)

        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        logname = os.path.join(dir_name, 'log.txt')
        val_curve.append(np.mean(val_loss))
        log_table = {
          "train_loss" : np.mean(train_loss),
          "val_loss" : np.mean(val_loss),
          "epoch" : epoch,
          "lr" : lrate,
          "bestval" : best_val_loss,

        }
        with open(logname, 'a') as f:
            f.write('json_stats: ' + json.dumps(log_table) + '\n')

        print('saving net...')
        torch.save(model.state_dict(), '%s/network.pth' % (dir_name))
  except:
    del(dataset)
    del(dataloader)
    del(dataset_val)
    del(dataloader_val)
    raise

from copy import copy
now = datetime.datetime.now()

try:
  del(embedder)
except:
  pass
try:
  del(model)
except:
  pass

if False:
  embedder = make_embedder(args) if should_embed else None
  in_c = embedder.decoder[0].conv3.out_channels if should_embed else 256
  print(in_c)
  #args.model = "/content/drive/MyDrive/saved/model.py"
  model = make_model(in_c, args)
  cs = 0
  for name, param in model.named_parameters():
    if param.requires_grad:
      cs += param.nelement()
      print(name, cs)
  train(embedder, model,save_path, args)

/content/repo_folder/MSN-Point-Cloud-Completion


In [11]:
if first:
  ! pip install transforms3d
  %cd /content/repo_folder
  ! rm -r /content/repo_folder/https:
  ! git clone https://github.com/lynetcha/completion3d.git
  %cd /content/repo_folder/completion3d
%cd /content/repo_folder/completion3d
chamferdir = "/content/repo_folder/completion3d/pytorch/utils/chamfer"
try:
  print(chamfer)
except:
  chamfer = load(name="chamfer", sources=[f"{chamferdir}/chamfer_cuda.cpp",f"{chamferdir}/chamfer.cu"])
  print(chamfer)

class chamferFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, xyz1, xyz2):
        batchsize, n, _ = xyz1.size()
        _, m, _ = xyz2.size()

        dist1 = torch.zeros(batchsize, n)
        dist2 = torch.zeros(batchsize, m)

        idx1 = torch.zeros(batchsize, n).type(torch.IntTensor)
        idx2 = torch.zeros(batchsize, m).type(torch.IntTensor)

        dist1 = dist1.cuda()
        dist2 = dist2.cuda()
        idx1 = idx1.cuda()
        idx2 = idx2.cuda()

        chamfer.forward(xyz1, xyz2, dist1, dist2, idx1, idx2)
        ctx.save_for_backward(xyz1, xyz2, idx1, idx2)
        return dist1, dist2

    @staticmethod
    def backward(ctx, graddist1, graddist2):
        xyz1, xyz2, idx1, idx2 = ctx.saved_tensors
        graddist1 = graddist1.contiguous()
        graddist2 = graddist2.contiguous()

        gradxyz1 = torch.zeros(xyz1.size())
        gradxyz2 = torch.zeros(xyz2.size())

        gradxyz1 = gradxyz1.cuda()
        gradxyz2 = gradxyz2.cuda()
        chamfer.backward(xyz1, xyz2, gradxyz1, gradxyz2, graddist1, graddist2, idx1, idx2)
        return gradxyz1, gradxyz2

class chamferDist(nn.Module):
    def __init__(self):
        super(chamferDist, self).__init__()

    def forward(self, input1, input2):
        return chamferFunction.apply(input1, input2)

try:
  import h5py
except:
  ! pip install h5py
  import h5py

import pytorch._init_paths
sys.path.insert(1, '/content/repo_folder/MSN-Point-Cloud-Completion/')
sys.path.insert(1, '/content/repo_folder/completion3d/shared')
sys.path.insert(1, '/content/repo_folder/completion3d/shared/datasets')
sys.path.insert(1, '/content/repo_folder/completion3d/pytorch')

import torch
import time
import numpy as np
import json
import logging
from tqdm import tqdm
import torch.optim as optim
from torch.autograd import Variable
import os
from collections import defaultdict
import h5py
from multiprocessing import Queue
from data_process import kill_data_processes
from shapenet import ShapenetDataProcess
import subprocess

def data_setup(args, phase, num_workers, repeat):
    if args.dataset == 'shapenet':
        DataProcessClass = ShapenetDataProcess
    # Initialize data processes
    data_queue = Queue(4 * num_workers)
    data_processes =[]
    for i in range(num_workers):
        data_processes.append(DataProcessClass(data_queue, args, phase, repeat=repeat))
        data_processes[-1].start()
    return data_queue, data_processes

def test(split, args, its = 100):
    """ Evaluated model on test set """
    print("Testing....")
    args.model.eval()

    data_queue, data_processes = data_setup(args, split, num_workers=1, repeat=False)
    losses = []
    N = len(data_processes[0].data_paths)
    Nb = int(N/args.batch_size)
    if Nb*args.batch_size < N:
        Nb += 1
    # iterate over dataset in batches
    Nb = min(Nb,its)
    for bidx in tqdm(range(Nb),total=Nb, position=0, leave=True):
      targets, clouds_data = data_queue.get()

      loss, dist1, dist2, emd_cost, outputs = args.step(targets, clouds_data)

      losses.append(loss)

    kill_data_processes(data_queue, data_processes)
    return np.mean(losses)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.0 MB/s eta 0:00:00
/content/repo_folder
rm: cannot remove '/content/repo_folder/https:': No such file or directory
Cloning into 'completion3d'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 194 (delta 3), reused 0 (delta 0), pack-reused 181
Receiving objects: 100% (194/194), 1.31 MiB | 27.98 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/repo_folder/completion3d
/content/repo_folder/completion3d
<module 'chamfer' from '/root/.cache/torch_extensions/py310_cu121/chamfer/chamfer.so'>


In [12]:
%load_ext autoreload
%autoreload 2
%cd /content/repo_folder/completion3d
def test_on_completion3D(args, download):
  dir = "/content/repo_folder/completion3d/data"
  test_data_queue, test_data_processes = data_setup(args, 'test', args.nworkers,
                                                        repeat=True)
  return test('train',args,50)



def make_base_model(args):
  model = MSN()
  network = FullModel(model)
  if args.base_model != '':
    previous_state_dict = torch.load(args.base_model)
    model.load_state_dict(previous_state_dict,False,True)
    print("Previous weight loaded ")
  return network

class completionargs:
  def __init__(self,model, batch_size):
    self.model = model
    self.batch_size = batch_size
    self.nworkers = 2
    self.dataset = 'shapenet'
    self.pc_augm_scale=0
    self.pc_augm_rot=0
    self.pc_augm_mirror_prob=0
    self.pc_augm_jitter=0
    self.inpts=500
  def step(self, targets, clouds_data):
    clouds_data = torch.from_numpy(clouds_data[1]).cuda()
    targets = torch.from_numpy(targets).cuda()
    output1, output2, emd1, emd2, expansion_penalty = self.model(clouds_data, targets.contiguous(), 0.005, 10)
    dist1, dist2 = chamferDist()(output2.float(),targets.float())
    loss = torch.mean(dist2) + torch.mean(dist1)
    return loss.item(), dist1, dist2, emd2, output2.detach().cpu().numpy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/repo_folder/completion3d


In [ ]:
train = False
class opt:
  def __init__(self):
    self.base_model = '/content/repo_folder/MSN-Point-Cloud-Completion/trained_model/network.pth'
    self.epoch_iter_limit_val = 10
    self.batchSize = 16
    self.epoch_iter_limit = None
    self.perc_data = 0.1 if train else 1
    self.perc_val_data = 0.1 if train else 0.999
    self.workers = 1
    self.nepoch = 10
    self.model = ''
    self.num_points = 8192
    self.n_primitives = 16
    self.env = "MSN_TRAIN"
    self.manualSeed = 0
    self.run_embedder = True
    self.embeddingpath = "/content/embeddings"

class architect_opts:
  def __init__(self, latent_size = 64, gt_layer_sizes = (32,128,512), res_layer_sizes = (256,128,64), use_gt = True, train = train):
    self.use_gt = use_gt
    self.gt_layer_sizes = (32,128,512)
    self.latents = 64
    self.res_layer_sizes = (256,128,64)
    self.train = train

arcargsd = {}

argsd = {}
arcargsd = {}
arcargsd["m1"] = architect_opts(None,None,(512,256,128,64),False,False)
#arcargsd["m2"] = architect_opts(None,None,(1024),False,False)
#arcargsd["m3"] = architect_opts(None,None,(512),False,False)
#arcargsd["m4"] = architect_opts(None,None,(256,64),False,False)
#arcargsd["m5"] = architect_opts(None,None,(512),False,False)
arcargsd["m6"] = architect_opts(32,(32,256,512),(512,256,128,64))
arcargsd["m7"] = architect_opts(64,(1024),(1024))
arcargsd["m8"] = architect_opts(64,(512),(512))
arcargsd["m9"] = architect_opts(64,(128,512),(256,64))
arcargsd["m10"] = architect_opts(64,(512,512),(512))


names = arcargsd.keys()

for name in names:
  argsd[name] = opt()

%cd /content/repo_folder/MSN-Point-Cloud-Completion/
if train:
  for name in names:
    if arcargsd[name].train:
      save_path = f"/content/drive/MyDrive/saved/{name}"
      model = make_fuller_model("", argsd[name], arcargsd[name])
      trainFull(model,save_path, False, argsd[name])

base_model = make_base_model(opt())
def print_results_sorted(results):
  vs = [(x,results[x]) for x in results]
  print([vs[x] for x in np.argsort([x[1] for x in vs])])
first = True
if False:
  %cd /content/repo_folder 3D-FUTURE-AI-Challenge-Baseline
  if first:
    ! python ./reconstruction/generate.py
  def testfunc(x):
    ca = completionargs(model,32)
    def trainer(data):
      ca.step(data[1],data[0])
    return eval(x, trainer)
else:
  if first:
    dir = "/content/repo_folder/completion3d/data"
    link = "http://download.cs.stanford.edu/downloads/completion3d/dataset2019.zip"
    dataf = "/content/data.zip"
    ! wget -nc -O $dataf $link
    ! rm -r $dir
    ! mkdir -p $dir
    ! unzip -o $dataf -d $dir
  %cd /content/repo_folder/completion3d
  testfunc = lambda x : test_on_completion3D(completionargs(x,32),False)

results = {}
if False:
  model = make_base_model(args)
  model.cuda()
  results["base_model"] = testfunc(model)
  print_results_sorted(results)

for name in list(names):
  save_path = f"/content/drive/MyDrive/saved/{name}"
  save_file = f'{save_path}/network.pth'
  if not os.path.isfile(save_file):
    continue
  model = make_fuller_model(save_file, argsd[name], arcargsd[name])
  model.cuda()
  results[name] = testfunc(model)
  print_results_sorted(results)
  del(model)

/content/repo_folder/MSN-Point-Cloud-Completion
Previous weight loaded 
--2024-01-31 14:08:56--  http://download.cs.stanford.edu/downloads/completion3d/dataset2019.zip
Resolving download.cs.stanford.edu (download.cs.stanford.edu)... 171.64.64.22
Connecting to download.cs.stanford.edu (download.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1585860897 (1.5G) [application/zip]
Saving to: ‘/content/data.zip’

/content/data.zip    51%[=========>          ] 774.77M  4.91MB/s    eta 2m 26s 


| model name  | loss * 1000  |   |   |   |
|---|---|---|---|---|
|  new res non trained | 22  |   |   |   |
| global transform new res trained 16 epochs | 16  |   |   |   |
| original | 9  |   |   |   |